In [1]:
# This is a step needed in jupyetr notebooks
# prepackaging of psif_lib library. After that it is not needed
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [2]:
import xarray as xr
from psif_lib import data_access
import psif_lib.wind_utils as wu
import psif_lib.processing as pr
import psif_lib.geo_helpers as geo

In [3]:
import os
import pandas as pd

def project_root():
    # Handles both script (__file__) and notebook (cwd) context
    if '__file__' in globals():
        return os.path.dirname(os.path.abspath(__file__))
    else:
        return os.path.abspath(os.path.join(os.getcwd(), '..'))

PROJECT_ROOT = project_root()
DATA_DIR = os.path.join(PROJECT_ROOT, 'data')

# Usage
#fn = os.path.join(DATA_DIR, 'daily_wind_Nebraska_statistics_2019.nc')
#print(fn)  # Always absolute and correct!


In [4]:
winds_path = os.path.join(DATA_DIR, 'winds')
winds = data_access.combine_netcdf_files(winds_path)

Found 8 NetCDF files to combine in '/Users/babak.jfard/projects/PSIF/data/winds':
  - daily_wind_Nebraska_statistics_2016.nc
  - daily_wind_Nebraska_statistics_2017.nc
  - daily_wind_Nebraska_statistics_2018.nc
  - daily_wind_Nebraska_statistics_2019.nc
  - daily_wind_Nebraska_statistics_2020.nc
  - daily_wind_Nebraska_statistics_2021.nc
  - daily_wind_Nebraska_statistics_2022.nc
  - daily_wind_Nebraska_statistics_2023.nc

Combined dataset information:
Dimensions: {'time': 1010, 'lat': 40, 'lon': 91}
Time range: 2016-01-29T00:00:00.000000000 to 2023-06-03T00:00:00.000000000


/Users/babak.jfard/projects/PSIF/psif_lib/data_access.py:55: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"Dimensions: {dict(combined_ds.dims)}")


In [5]:
fires_path = os.path.join(DATA_DIR, 'fires', 'fire_archive_M-C61_581147.csv')
fires = pd.read_csv(fires_path)

In [6]:
fires_with_wind = wu.get_wind_at_fire(fires=fires, ds_wind=winds)

In [7]:
# Making sure to include fires in the required time period
fires_with_wind = pr.filter_by_month_day(fires_with_wind,
                                     date_col="acq_date",
                                     start_md="01-29",
                                     end_md="06-01")


In [8]:
# Get the Block Group Centroids
BG_path = os.path.join(DATA_DIR, 'aux', 'NE-BlockGroups-Centers_of_population.csv')
BGs = pd.read_csv(BG_path)

In [9]:
# Creating the GEOID for the BGs
pieces = [
    BGs['STATEFP'].astype(str).str.zfill(2),
    BGs['COUNTYFP'].astype(str).str.zfill(3),   # pad to 3
    BGs['TRACTCE'].astype(str).str.zfill(6),
    BGs['BLKGRPCE'].astype(str).str.zfill(1)
]
BGs["GEOID"] = pd.concat(pieces, axis=1).agg("".join, axis=1)
BGs.drop(columns=['STATEFP', 'COUNTYFP', 'TRACTCE', 'BLKGRPCE'], inplace=True)

In [10]:
fires_with_wind = pr.prep_fires(fires_with_wind)
BGs.rename(columns={'LATITUDE':'latitude', 'LONGITUDE': 'longitude'}, inplace=True)
BGs_final = pr.prep_bgs(BGs)


pairs = pr.fire_bg_pairs(fires_with_wind, BGs_final)
# adding the wind probabilities at BG
pairs.rename(columns={'latitude_bg': 'latitude', 'longitude_bg': 'longitude'}, inplace=True)
pairs = wu.get_wind_at_fire(winds, pairs, "_bg")

PSIF_BG_level = pr.psif_from_pairs(pairs)

In [11]:
PSIF_with_moving = pr.calculate_moving_average(PSIF_BG_level, 'PSIF', 'GEOID','01/28', '05/31')

In [17]:
PSIF_with_moving['PSIF_Total'] = PSIF_with_moving['PSIF']+ PSIF_with_moving['moving_avg']

In [18]:
PSIF_with_moving.to_csv('../data/output_test/PSIF_BGs_NE.csv')

In [19]:
# Converting into zcta level
BG_to_ZCTA_map = pd.read_csv('../data/aux/Nebraska_BG_zcta_crosswalk.csv')
BG_to_ZCTA_map.drop(index=0, inplace=True)

def format_tract(tract):
    # Remove decimal and pad to 6 digits
    tract_str = str(tract)
    if '.' in tract_str:
        left, right = tract_str.split('.')
        right = right.ljust(2, '0')  # Ensure two digits after decimal
        tract_str = left.zfill(4) + right
    else:
        tract_str = tract_str.zfill(6)
    return tract_str

BG_to_ZCTA_map['tract_str'] = BG_to_ZCTA_map['tract'].apply(format_tract)

# county: pad to 3 digits, blockgroup: as string
BG_to_ZCTA_map['county_str'] = BG_to_ZCTA_map['county'].astype(str).str.zfill(3)
BG_to_ZCTA_map['blockgroup_str'] = BG_to_ZCTA_map['blockgroup'].astype(str)

# Now construct GEOID
BG_to_ZCTA_map['GEOID'] = (
    BG_to_ZCTA_map['county_str'] +
    BG_to_ZCTA_map['tract_str'] +
    BG_to_ZCTA_map['blockgroup_str']
)

In [20]:
BG_to_ZCTA_map['afact'] = pd.to_numeric(BG_to_ZCTA_map['afact'], errors='coerce')
BG_to_ZCTA_map['afact'] = BG_to_ZCTA_map['afact'].astype('float64')

In [21]:
# Now you can merge as before
merged = pd.merge(PSIF_with_moving, BG_to_ZCTA_map[['GEOID', 'zcta', 'afact']], on='GEOID', how='left')
merged['weighted_psif'] = merged['PSIF_Total'] * merged['afact']
#merged['weighted_mav'] = merged['moving_avg'] * merged['afact']


zcta_psif = (
    merged.groupby(['zcta', 'acq_date'], as_index=False)
    .agg(
        zcta_psif=('weighted_psif', 'sum')
    ))

In [22]:
PSIF_with_moving.to_csv('../data/output_test/Nebraska_BlockGroup_PSIF.csv', index=False)